In [1]:
import torch
import transformers
import tokenizers
from tokenizers import Tokenizer
import pprint
from transformers import PreTrainedTokenizerFast
from transformers import BertConfig, BertLMHeadModel
import numpy as np
import json
import pickle
from astTokenizer import CustomTokenizer

/home/aktarafder/Documents/SoftwareProject/HF/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def calculate_distance(n1, n2):
    """
    takes 2 numpy array
    n1, n2, each a vector embedding for a single code_string 
    """
    euclidean_dist=np.linalg.norm(n1-n2)
    #print(n1==n2)
    manhattan_dist=np.sum(np.abs(n1 - n2))
    cosine_sim=np.dot(n1,n2)/(np.linalg.norm(n1)*np.linalg.norm(n2))
    dot_product=np.dot(n1,n2)

    return euclidean_dist, manhattan_dist, cosine_sim, dot_product

In [3]:
edistances = []
cosSims = []
dotPros = []

In [4]:
tokenizer=Tokenizer.from_file('./combined_tokenizer')
custom_t=CustomTokenizer(tokenizer)
#fast_tokenizer = PreTrainedTokenizerFast(tokenizer_object=custom_t)
#fast_tokenizer.mask_token='<mask>'
#fast_tokenizer.pad_token='<pad>'
# Load the pre-trained BERT model and tokenizer
model = BertLMHeadModel.from_pretrained('../ast_transformer/checkpoint-12400/',output_hidden_states=True)
        

In [5]:
# read scd-88 data
with open('../data/all.jsonl','r') as test_file:
    test_list = list(test_file)

In [6]:
code=json.loads(test_list[1*130])['code']

In [7]:
print(len(custom_t.encode(code)[:512]))
print(type(torch.tensor((custom_t.encode(code)[:512]))))

52
<class 'torch.Tensor'>


In [8]:
euclids=list()
cosines=list()
dots=list()
manhattans=list()

with torch.no_grad():
    
    for i in range(0,88,1):
        pairs=list()

        p_euclid=list()
        p_cosine=list()
        p_dot=list()
        p_manhattan=list()

        for p in range(2,102,1):
            clone1 = json.loads(test_list[i*130])['code']
            clone2 = json.loads(test_list[i*130+p])['code']

            pairs.append([clone1, clone2])

            #print(clone1)
            #print(clone2)
            #print('---------')

        # get embedding
        for code_pair in pairs:
            code1 = code_pair[0]
            code2 = code_pair[1]
        
            # Tokenize the input sentence
            #id1 = fast_tokenizer.encode(code1, return_tensors='pt', max_length=512, truncation=True)
            #id2 = fast_tokenizer.encode(code2, return_tensors='pt', max_length=512, truncation=True)

            id1= torch.tensor((custom_t.encode(code1)[:512]))
            id2= torch.tensor((custom_t.encode(code2)[:512]))

            # unsqueeze is necessary as our custom_tokenizer returns different shaped lists
            # this converts [dim] to [1,dim] for single peace of code, which we are using here 
            id1=id1.unsqueeze(0)
            id2=id2.unsqueeze(0)

            #print(id1)
            #print(type(id1))
            #print(id1.shape)
            
            out_clone1 = model(id1)
            out_clone2 = model(id2)

            v1=torch.zeros(out_clone1.hidden_states[0].shape)# v1, v2 must match the shape
            v2=torch.zeros(out_clone2.hidden_states[0].shape)

            for i in range(len(out_clone1.hidden_states)):# we have 3 layers
                v1 += out_clone1.hidden_states[i]
            
            mean_embed_clone1 = torch.mean(v1, dim=1).squeeze()

            for i in range(len(out_clone2.hidden_states)):# we have 3 layers
                v2 += out_clone2.hidden_states[i]
            
            mean_embed_clone2 = torch.mean(v2, dim=1).squeeze()
            #print(mean_embed_clone2.shape)

            e,m,c,d = calculate_distance(mean_embed_clone1.numpy(), mean_embed_clone2.numpy())
            #print(m)
            p_euclid.append(e)
            p_cosine.append(c)
            p_dot.append(d)
            p_manhattan.append(m)

        euclids.append(p_euclid)
        cosines.append(p_cosine)
        dots.append(p_dot)
        manhattans.append(p_manhattan)


In [9]:
list(id1)

[tensor([13005,  5971, 36646,  5971, 27207, 20429, 11172, 16228, 44446, 16228,
         25702,  5971, 20260, 16228, 30587, 39189, 11065, 16228, 30587, 30587,
         30587, 45933, 31975, 13005, 31045, 36646,  8995, 45933, 45932, 45932,
         45932, 45932, 22454, 16228, 27207, 30587, 45933, 12796, 13005, 31045,
         27207,  8995, 45933, 45932, 45932, 45932, 45932, 22454, 16228, 36646,
         30587, 45933, 30834,  8995, 45933, 45932, 45932, 45932, 45932, 22454,
         16228, 13005, 30587])]

In [93]:
with open('euclids_combined.pkl', 'wb') as f:
    pickle.dump(euclids, f)

with open('cosines_combined.pkl', 'wb') as f:
    pickle.dump(cosines, f)

with open('dots_combined.pkl', 'wb') as f:
    pickle.dump(dots, f)

with open('manhattans_combined.pkl', 'wb') as f:
    pickle.dump(manhattans, f)